# Export Earth Engine data

In [1]:
import shapely.wkb 
import LMIPy
import time
import ee
ee.Initialize()

/Users/ikersanchez/anaconda3/envs/geoenv/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  warnings.warn(


### Earth Engine datasets

**USA geometry**

In [2]:
params={
    'iso': 'USA',
    'adm1': None,
    'adm2': None
}
geometry = LMIPy.Geometry(parameters=params)

### Datasets

In [8]:
datasets = {"dis007-Landslide-Susceptibility_replacement": "ea2db3a6-49c8-4d41-a2ab-758eb6fe4bc0",
           "dis004-Earthquake-Frequency-and-Distribution": "b3ebc10d-9de8-4ee6-870d-1d049e8e2a99",
           "dis008-Volcano-Frequency-and-Distribution": "74ebf4fe-6afb-46aa-a820-96fcae9a7e03",
           "bio015-Terrestrial-Human-Footprint_replacement": "7f7bf238-e820-48c4-abb2-678bc970225b",
           "bio_014-Biodiversity-Intactness": "0e565ddf-74fd-4f90-a6b8-c89d747a89ab"}

**Example**

In [11]:
ds = LMIPy.Dataset(list(datasets.values())[0])
ds

Dataset ea2db3a6-49c8-4d41-a2ab-758eb6fe4bc0 dis.007 Landslide Susceptibility_replacement

In [16]:
ds.layers[0].map(geometry=geometry)

**Export images to Cloud Store**

In [26]:
for slug, id in datasets.items():
    ds = LMIPy.Dataset(id)    
    
    name = ds.attributes.get('name').replace(" ", "_")
    print(f"Export {name}!")
    
    assetId = ds.attributes.get('tableName')
    image = ee.Image(assetId )
    region = ee.Geometry.Rectangle([-179.1305542, 18.9098587, -55., 71.3354187])
    
    bands = image.bandNames().getInfo()
    scale = int(image.select(bands[0]).projection().nominalScale().getInfo())
    
    bucket = 'us-resilience-map'
    task = ee.batch.Export.image.toCloudStorage(
        image = image.unmask(-9999),
        description = f'Exporting {name}', 
        bucket=bucket,
        fileNamePrefix = f'datasets/{name}',
        scale = scale, 
        region = region.getInfo().get('coordinates'),
        maxPixels = 1e13,
        fileFormat = 'GeoTIFF'
    )
    task.start()
    
    status = task.status().get('state')
    while not status in ['COMPLETED', 'FAILED']:
        status = task.status().get('state')
        print('Temporal status: ', status)
    
        time.sleep(60)
    print('Final status: ', status)

Export dis.007_Landslide_Susceptibility_replacement!
Export dis.004_Earthquake_Frequency_and_Distribution!
Export dis.008_Volcano_Frequency_and_Distribution!
Export bio.015_Terrestrial_Human_Footprint_replacement!
Export bio.014_Biodiversity_Intactness!


**Export PREP LOCA images to Cloud Store**

In [10]:
assetIds = {'cli.056a_Projected_Change_in_Extreme_Heat_Days': 'projects/resource-watch-gee/loca/loca_gt-q99_tasmax/loca_rcp45_gt-q99_tasmax_diff_2015_2045',
            'cli.059a_Projected_Change_in_Extreme_Precipitation_Days': 'projects/resource-watch-gee/loca/loca_gt-q99_pr/loca_rcp45_gt-q99_pr_ch_2015_2045'}

for name, assetId in assetIds.items():
    print(f"Export {name}!")
    
    image = ee.Image(assetId )
    region = ee.Geometry.Rectangle([-179.1305542, 18.9098587, -55., 71.3354187])
    
    bands = image.bandNames().getInfo()
    
    scale = int(image.select(bands[1]).projection().nominalScale().getInfo())
    
    bucket = 'us-resilience-map'
    task = ee.batch.Export.image.toCloudStorage(
        image = image.select(bands[1]),
        description = f'Exporting {name}', 
        bucket=bucket,
        fileNamePrefix = f'datasets/{name}',
        scale = scale, 
        region = region.getInfo().get('coordinates'),
        maxPixels = 1e13,
        fileFormat = 'GeoTIFF'
    )
    task.start()
    
    status = task.status().get('state')
    while not status in ['COMPLETED', 'FAILED']:
        status = task.status().get('state')
        print('Temporal status: ', status)
    
        time.sleep(60)
    print('Final status: ', status)

Export cli.056a_Projected_Change_in_Extreme_Heat_Days!
Temporal status:  READY
Temporal status:  COMPLETED
Final status:  COMPLETED
Export cli.059a_Projected_Change_in_Extreme_Precipitation_Days!
Temporal status:  READY
Temporal status:  COMPLETED
Final status:  COMPLETED
